In [1]:
import numpy as np

In [2]:
d = 10 # Dimensions of content

# Initialize pool of content Ct
content_pool = []

num_iterations = 1000

num_bins = 5
bin_boundaries = np.linspace(0, 10, num_bins + 1)

# Initialize parameters βi and uncertainties ui
beta_parameters = np.zeros(num_bins)# initialize beta parameters
uncertainties = np.zeros(num_bins)# initialize uncertainties

In [3]:
def scale_risk_score(content):
    return f(beta_parameters, uncertainties,content)

def f(beta_parameters, uncertainties,content):
    result = np.zeros_like(content)
    for j in range(num_bins):
        bin_mask = np.logical_and(bin_boundaries[j] <= content, content < bin_boundaries[j + 1])
        a_ij, b_ij = beta_parameters[j] + uncertainties[j]
        result[bin_mask] = a_ij * content[bin_mask] + b_ij
    return result
    

def update_beta_parameters(beta_parameters, risk_score, true_severity):
    for j in range(num_bins):
        bin_mask = np.logical_and(bin_boundaries[j] <= risk_score, risk_score < bin_boundaries[j + 1])
        x_j = risk_score[bin_mask]  # Extract the j-th column of X (risk scores for bin j)
        xtj_sq_sum = np.sum(x_j ** 2)  # Sum of squares of risk scores for bin j
        xtj_yt_sum = np.sum(x_j * true_severity)  # Sum of product of risk scores for bin j and predicted severity
        
        if xtj_sq_sum != 0:  # Avoid division by zero
            beta_parameters[j] = xtj_yt_sum / xtj_sq_sum  # Update beta parameter for bin j
    
    return beta_parameters

def update_uncertainties(uncertainties, risk_score):
    for j in range(num_bins):
        delta = np.random.random_sample()
        bin_mask = np.logical_and(bin_boundaries[j] <= risk_score, risk_score < bin_boundaries[j + 1])
        x_j = risk_score[bin_mask]  # Extract the j-th column of X (risk scores for bin j)
        xtj_sq_sum = np.sum(x_j ** 2)  # Sum of squares of risk scores for bin j
        
        if xtj_sq_sum != 0:  # Avoid division by zero
            uncertainties[j] = np.sqrt(np.log(delta)/ xtj_sq_sum)  # Update beta parameter for bin j
    
    return uncertainties

In [4]:
for t in range(num_iterations):
    
    content = np.random.uniform(0, 10, size=d)
    content_pool.append(content)
    
    # Iterate through content in pool to find content to review
    max_risk_score = -np.inf
    content_to_review = None
    for con in content_pool:
        risk_score = scale_risk_score(con)
        if risk_score > max_risk_score:
            max_risk_score = risk_score

    if max_risk_score>0:
        content_to_review = con
    
        # Review the selected content
        true_severity = np.random.uniform(0, 10, size=d) # Manual Review (Random for this simulation)
        
        beta_parameters = update_beta_parameters(beta_parameters, max_risk_score, true_severity)
        uncertainties = update_uncertainties(uncertainties, max_risk_score)

TypeError: cannot unpack non-iterable numpy.float64 object